# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Шаг 1. Откройте файл с данными и изучите общую информацию

In [8]:
#подключим необходимую библитеку
import pandas as pd
from pymystem3 import Mystem
from collections import Counter
#создадим переменную, в которой будем хранить набор данных
data = pd.read_csv('/datasets/data.csv')
#отобразим набор данных, для изучения (первых 15 строк)
display(data.head(15))
data.info()
data.describe()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,1.935100e+04
mean,0.538908,63046.497661,43.293380,0.817236,0.972544,0.080883,1.674223e+05
std,1.381587,140827.311974,12.574584,0.548138,1.420324,0.272661,1.029716e+05
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,1.030532e+05
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,1.450179e+05
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,2.034351e+05
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


**Вывод**

 В наборе данных отсутствуют некоторые значения в столбцах : *days_employed* (общий трудовой стаж) , *total_income*(ежемесячный доход),вместо значений указано значение **Nan**.Ещё в данных замечен артефакт,общий трудовой стаж(*days_employed*) указан отрицательным(может так и должно быть?).

## Шаг 2. Предобработка данных

### Обработка пропусков

In [9]:
#Подсчитаем сколько раз было  пропущено значение общего трудового стажа(days_employed) 
print('в ячейке','*','days_employed','*','-',len(data[data['days_employed'].isna()]),'пропуска')
#Подсчитаем количество пропусков значений в столбце *total_income*
print('в ячейке','*','total_income','*','-',len(data[data['total_income'].isna()]),'пропуска')
print()
#Убедимся в отсутствии любых других пропусков в наборе данных
print(data.isna().sum())
print()
#подсчитаем количество % пропусков от общего числа данных, чтобы сделать дальнейшие выводы
print((len(data[data['total_income'].isna()]) /21525) *100 )
#Поскольку пренебрегать таким количеством пропусков нельзя, заменим пропуски медианным значением
data['days_employed'] = data['days_employed'].abs()
#data.loc[data['days_employed'].isna(),'days_employed'] = data['days_employed'].median()
data.loc[data['days_employed'] > 300000 , 'days_employed'] = data.loc[data['days_employed'] > 300000, 'days_employed'] / 24 
data.loc[data['days_employed'].isna(),'days_employed'] = data['days_employed'].median()

data['total_income'] = data['total_income'].abs()
data.loc[data['total_income'].isna(),'total_income'] = data['total_income'].median()
print()
#Выполним проверку по замене данных и распечатаем первые 15 значений
print('Пропуски после замены :',data['days_employed'].isna().sum())
print()
print('Пропуски после замены :',data['total_income'].isna().sum())
display(data.head(15))

в ячейке * days_employed * - 2174 пропуска
в ячейке * total_income * - 2174 пропуска

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

10.099883855981417

Пропуски после замены : 0

Пропуски после замены : 0


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,14177.753002,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


**Вывод**

 Проанализировав полученные данные , можно сделать вывод, что явные пропуски присутствуют всего в двух столбцах( *общей трудовой стаж,ежемесячный доход* и имеют значение Nan(т.е вещественные числа, поэтому был выбран данный метод для поиска пропуска значений) и составляют чуть более **10** % от всего объема данных, соотвественно пренебрегать этими пропусками **нежелательно!!!**.Поэтому необходимо было произвести замену значений , чтобы было можно продолжить работу с данными.Так же были замечены и ошибки связанные с человеческим фактором(ошибка при наборе данных) и ошибки связанные с неоднотипным значением(то есть трудовой стаж измеряется не только в днях, но и часах наверное)

In [10]:
#в предыдущем коде нашли явные пропуски, а что если имеются еще и ошибки допущенные оператором?
print(data.groupby('gender')['gender'].count())
print()
print(data['dob_years'].unique())
print()
print(data.groupby('children')['children'].count())
print()

#Найдя явные ошибки в данных ,сделанные оператором, попробуем их исправить

#применим функцию замены значений для пола, заменив неподходящее значение на наиболее встречаемый пол
data['gender'] = data['gender'].replace('XNA',data['gender'].value_counts().idxmax())
print(data.groupby('gender')['gender'].count())
print()
#аналогично выполним и для устранения артефактов в столбце дети(children)
data['children'] = data['children'].replace(-1, 1)
data['children'] = data['children'].replace(20, 2)
print(data.groupby('children')['children'].count())

gender
F      14236
M       7288
XNA        1
Name: gender, dtype: int64

[42 36 33 32 53 27 43 50 35 41 40 65 54 56 26 48 24 21 57 67 28 63 62 47
 34 68 25 31 30 20 49 37 45 61 64 44 52 46 23 38 39 51  0 59 29 60 55 58
 71 22 73 66 69 19 72 70 74 75]

children
-1        47
 0     14149
 1      4818
 2      2055
 3       330
 4        41
 5         9
 20       76
Name: children, dtype: int64

gender
F    14237
M     7288
Name: gender, dtype: int64

children
0    14149
1     4865
2     2131
3      330
4       41
5        9
Name: children, dtype: int64


In [11]:
# КОД РЕВЬЮВЕРА

data['dob_years'] = data.groupby('family_status')['dob_years'].transform(lambda x: x.replace(0, int(x.median())))

### Замена типа данных

 Для дальнейшей работы необходимо произвести некоторые преобразования типов данных, в частности применить преобразование для двух столбцов(так как например *общий трудовой стаж* не может явно измеряться в частях дней).Для начала необходимо определиться с методом преобразования данных.Наиболее всего подходит метод astype, так как необходимо произвести преобразование из вещественного типа данных в целочисленный.

In [12]:
#Проверим метод преобразования для двух столбцов (days_employed,total_income  )
data['total_income'] = data['total_income'].astype(int)
data['days_employed'] = data['days_employed'].astype(int)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null int64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null int64
purpose             21525 non-null object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


**Вывод**

Выполнив преобразование данных, необходимо убедиться в отсутствии дубликатов.

### Обработка дубликатов

In [13]:
#Найдем явные дубликаты и их количество
print(data.duplicated().sum())
#Посмотрим на дубликаты в табличном виде
duplicated_data = data[data.duplicated(keep = False)].sort_values(by = list(data.columns))
display(duplicated_data)
#Приведем к единому регистру столбец с дубликатами
data['education'] = data['education'].str.lower()
print(data['education'].value_counts())
data = data.drop_duplicates().reset_index(drop=True)
display(data)

55


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
15892,0,2194,23,среднее,1,Не женат / не замужем,4,F,сотрудник,0,145017,сделка с подержанным автомобилем
19321,0,2194,23,среднее,1,Не женат / не замужем,4,F,сотрудник,0,145017,сделка с подержанным автомобилем
3452,0,2194,29,высшее,0,женат / замужем,0,M,сотрудник,0,145017,покупка жилой недвижимости
18328,0,2194,29,высшее,0,женат / замужем,0,M,сотрудник,0,145017,покупка жилой недвижимости
520,0,2194,35,среднее,1,гражданский брак,1,F,сотрудник,0,145017,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
9238,2,2194,34,среднее,1,женат / замужем,0,F,сотрудник,0,145017,покупка жилья для сдачи
9013,2,2194,36,высшее,0,женат / замужем,0,F,госслужащий,0,145017,получение образования
14432,2,2194,36,высшее,0,женат / замужем,0,F,госслужащий,0,145017,получение образования
11033,2,2194,39,среднее,1,гражданский брак,1,F,сотрудник,0,145017,сыграть свадьбу


среднее                15233
высшее                  5260
неоконченное высшее      744
начальное                282
ученая степень             6
Name: education, dtype: int64


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,14177,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
21448,1,4529,43,среднее,1,гражданский брак,1,F,компаньон,0,224791,операции с жильем
21449,0,14330,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999,сделка с автомобилем
21450,1,2113,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672,недвижимость
21451,3,3112,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093,на покупку своего автомобиля


**Вывод**

 Были найдены и удалены 71 явный дубликат. Дубликаты появились скорее всего по причине ошибки человека-оператора, который заполнял заявки на одобрение кредитов.

### Лемматизация

In [14]:
#выделим уникальные значения в целях приобретения кредита
purpose_str = data['purpose'].unique()
print(",".join(purpose_str))
print()
#подсчитаем количество уникальных значений
print(Counter(purpose_str))
#выделим основные группы,лемы("образование","недвижимость","автомобиль","прочее")
#Создадим функцию, которая будет последовательно проходить по всему списку и в зависимости от лемматизации будет выдавать значение
m = Mystem()
def categorize(purpose):
    purpose_lemmas = m.lemmatize(purpose)
    if 'недвижимость' in purpose_lemmas or 'жилье' in purpose_lemmas:
        return 'недвижимость'
    if 'автомобиль' in purpose_lemmas:
        return 'автомобиль'
    if 'образование' in purpose_lemmas:
        return 'образование'
    if 'свадьба' in purpose_lemmas:
        return 'свадьба'
    return 'прочее'

data['category'] = data['purpose'].apply(categorize)
display(data.head(5))

покупка жилья,приобретение автомобиля,дополнительное образование,сыграть свадьбу,операции с жильем,образование,на проведение свадьбы,покупка жилья для семьи,покупка недвижимости,покупка коммерческой недвижимости,покупка жилой недвижимости,строительство собственной недвижимости,недвижимость,строительство недвижимости,на покупку подержанного автомобиля,на покупку своего автомобиля,операции с коммерческой недвижимостью,строительство жилой недвижимости,жилье,операции со своей недвижимостью,автомобили,заняться образованием,сделка с подержанным автомобилем,получение образования,автомобиль,свадьба,получение дополнительного образования,покупка своего жилья,операции с недвижимостью,получение высшего образования,свой автомобиль,сделка с автомобилем,профильное образование,высшее образование,покупка жилья для сдачи,на покупку автомобиля,ремонт жилью,заняться высшим образованием

Counter({'покупка жилья': 1, 'приобретение автомобиля': 1, 'дополнительное образование': 1, 'сыграть свадьбу': 1, 'опера

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,category
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,недвижимость
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,автомобиль
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,недвижимость
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,образование
4,0,14177,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,свадьба


**Вывод**

 Из полученных результатов можно сделать вывод, что люди в основном берут кредиты на 3 категории : Недвижимость(различные операции с ней :покупка, сдача в аренду), Образование , Автомобиль, Свадьба. На основе этих 3-х групп и были выделены леммы.

### Категоризация данных

In [15]:
#проведем категоризацию данных по образованию,для дальнейшего использования , чтобы выяснить влияет ли образование на возврат кредитов
education_group = data.groupby('education')['education'].count()
display(education_group)
#Проведем категоризацию по уровню дохода , для этого создадим функцию
def total_income_categorize(total_income):
    total_income_median = data['total_income'].median()
    if total_income > (total_income_median)*(2):
        return 'высокий'
    if total_income < (total_income_median) / (2):
        return 'низкий'
    return 'средний'

data['total_income_categorize'] = data['total_income'].apply(total_income_categorize)
display(data.head(5))

#Сделаем категоризацию по наличию детей в семьях следующим образом(0-бездетные,1-2-обычная семья, 3 и более -многодетная)
def children_categorize(children):
    if children == 0:
        return 'бездетная семья'
    if children <=2:
        return 'обычная семья'
    return 'многодетная семья'

data['children_categorize'] = data['children'].apply(children_categorize)
display(data.head(5))

education
высшее                  5250
начальное                282
неоконченное высшее      744
среднее                15171
ученая степень             6
Name: education, dtype: int64

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,category,total_income_categorize
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,недвижимость,средний
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,автомобиль,средний
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,недвижимость,средний
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,образование,средний
4,0,14177,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,свадьба,средний


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,category,total_income_categorize,children_categorize
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,недвижимость,средний,обычная семья
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,автомобиль,средний,обычная семья
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,недвижимость,средний,бездетная семья
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,образование,средний,многодетная семья
4,0,14177,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,свадьба,средний,бездетная семья


**Вывод**

 На основе полученных данных, можно с уверенностью сказать, что чем ниже уровень образования, тем больше берут кредит под различные цели.

In [16]:
# КОД РЕВЬЮВЕРА

data['total_income_categorize'].value_counts()

средний    18137
низкий      1661
высокий     1655
Name: total_income_categorize, dtype: int64

## Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [17]:
#Сгруппируем данные по кол-ву детей и подсчитаем вероятность не возврата кредита в %
children_report = data.groupby('children').agg({'debt' : ['count','sum','mean']})
children_report.columns = ['total','debt','%']
children_report

,total,debt,%
children,,,
0,14090,1063,0.075444
1,4855,445,0.091658
2,2128,202,0.094925
3,330,27,0.081818
4,41,4,0.097561
5,9,0,0.000000


**Вывод**

 Проанализировав полученные данные можно сделать вывод, что чем больше детей у заемщика, тем больше вероятность не возврата кредита.Хотя если посмотреть на семьи с 5ю детьми, то там 100% вероятность возврата, но думаю, это скорее из-за недостатка данных. 

In [18]:
# КОД РЕВЬЮВЕРА

data.groupby('children_categorize')['debt'].mean()

# видим, что наличие детей в любом случае повышает риск, однако если изучить кол-во, то видно, что малодетные семьи более рисковые,
# нежели многодетные

children_categorize
бездетная семья      0.075444
многодетная семья    0.081579
обычная семья        0.092654
Name: debt, dtype: float64

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [10]:
#сгруппируем данные по семейному положению и подсчитаем вероятность невозврата кредита
family_status_report = data.groupby('family_status').agg({'debt' : ['count','sum','mean']})
family_status_report.columns = ['total','debt','%']
family_status_report

,total,debt,%
family_status,,,
Не женат / не замужем,2810,274,0.097509
в разводе,1195,85,0.071130
вдовец / вдова,959,63,0.065693
гражданский брак,4151,388,0.093471
женат / замужем,12339,931,0.075452


**Вывод**

 Самая малая вероятность не возврата кредита у группы *вдовец/вдова* (может быть из-за того, что один из супругов был застрахован и являлся созаемщиком), самая большая вероятность не возврата кредита у группы *не женат/не замужем* и *гражданский брак* , т.е у холостых людей.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [11]:
#используя значения из категоризации по уровню заработка посчитаем зависимость
result_total_income = data.groupby('total_income_categorize').agg({'debt': ['count', 'sum', 'mean']})
result_total_income.columns = ['total', 'debt', '%']
result_total_income

,total,debt,%
total_income_categorize,,,
высокий,1655,120,0.072508
низкий,1661,115,0.069235
средний,18138,1506,0.083030


**Вывод**

 Из полученных данных видим, что самый низкий процент невозврата кредита у категории людей с низким уровнем заработка, а самый высокий процент со средним уровнем заработка.

- Как разные цели кредита влияют на его возврат в срок?

In [12]:
result_purpose = data.groupby('category').agg({'debt': ['count', 'sum', 'mean']})
result_purpose.columns = ['total', 'debt', '%']
result_purpose

,total,debt,%
category,,,
автомобиль,4306,403,0.093590
недвижимость,10811,782,0.072334
образование,4013,370,0.092200
свадьба,2324,186,0.080034


**Вывод**

 Самая высокая вероятность невозврата кредита возникает, когда люди берут его на покупку автомобиля и образование, самая низкая на операции с недвижимостью.

## Шаг 4. Общий вывод

 На основе работы с полученными данными можно сделать вывод, что самыми нежелательными клиентами по выдаче кредита у банка должны быть люди одинокие(не замужем/ не женат), если кредит будет браться на приобретение автомобиля и людей с большим количество детей. Может быть именно поэтому банки стали страховаться от не возврата кредита предлагая различного рода допы(страховка, крдеит под залог имущества, наличие созаемщика), а может просто это у них работа такая! ))).А вот кому действительно стоит выдавать кредит так это людям со средним уровнем заработка(опять же в рамках проекта),людям состоящим в зарегистрированном браке и либо с 1 ребенком либо вообще без детей.

## Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  открыт файл;
- [x]  файл изучен;
- [x]  определены пропущенные значения;
- [x]  заполнены пропущенные значения;
- [x]  есть пояснение, какие пропущенные значения обнаружены;
- [x]  описаны возможные причины появления пропусков в данных;
- [x]  объяснено, по какому принципу заполнены пропуски;
- [x]  заменен вещественный тип данных на целочисленный;
- [x]  есть пояснение, какой метод используется для изменения типа данных и почему;
- [x]  удалены дубликаты;
- [x]  есть пояснение, какой метод используется для поиска и удаления дубликатов;
- [x]  описаны возможные причины появления дубликатов в данных;
- [x]  выделены леммы в значениях столбца с целями получения кредита;
- [x]  описан процесс лемматизации;
- [x]  данные категоризированы;
- [x]  есть объяснение принципа категоризации данных;
- [x]  есть ответ на вопрос: "Есть ли зависимость между наличием детей и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Есть ли зависимость между семейным положением и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Есть ли зависимость между уровнем дохода и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Как разные цели кредита влияют на его возврат в срок?";
- [x]  в каждом этапе есть выводы;
- [x]  есть общий вывод.